# 데이터셋 불러오기

In [23]:
from dotenv import load_dotenv
import os

load_dotenv()

ROBOFLOW_API_KEY = os.environ.get('ROBOFLOW_API_KEY')

In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("hayley-kim-6vkvh").project("drug-detect-2-ssykt")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to drug-detect-2-1 in yolov11:: 100%|██████████| 2298/2298 [00:01<00:00, 1201.85it/s]


In [24]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("hayley-kim-6vkvh").project("drug-detect-cinr7")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to drug-detect-1 in yolov11:: 100%|██████████| 4162/4162 [00:05<00:00, 790.35it/s] 


# 모델 불러오기

In [ ]:
from ultralytics import YOLO
import torch

model = YOLO("./models/yolo11n.pt")

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [27]:
print(torch.__version__)
print(torch.version.cuda)   # CUDA 빌드 버전
print(torch.cuda.is_available())

2.8.0+cu126
12.6
True


# 모델 학습하기

In [26]:
import os
os.environ["ULTRALYTICS_TB"] = "1"  # 텐서보드 로거 활성화

In [30]:
PROJECT_NAME = "drug-detect-1"

In [ ]:
# 데이터셋 yaml 경로
DATA_YAML_PATH = f"./{PROJECT_NAME}/data.yaml"

In [31]:
# 학습
results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,
    batch=32,
    imgsz=640,
    device=device,
    project=f"runs/{PROJECT_NAME}",
    save=True,
    save_period=10,
    verbose=True
)

New https://pypi.org/project/ultralytics/8.3.200 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.198  Python-3.10.18 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./drug-detect-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Potenup\Drug-Detection-Chatbot\modeling\segment\runs\drug-detect2\weights\best.p

# 모델 평가하기

In [14]:
best_model_path = "C:\Potenup\Drug-Detection-Chatbot\modeling\segment\\runs\drug-detect2\weights\\best.pt"

In [15]:
load_model = YOLO(best_model_path)

In [ ]:
val_results = load_model.val(
    data=DATA_YAML_PATH, 
    imgsz=640, 
    iou=0.5, 
    save=True,
    project=f"runs/{PROJECT_NAME}",
    )

Ultralytics 8.3.198  Python-3.10.18 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 1214.0376.8 MB/s, size: 956.3 KB)
val: Scanning C:\Potenup\Drug-Detection-Chatbot\modeling\segment\drug-detect-2-1\valid\labels.cache... 198 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 198/198  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 3.7it/s 3.5s0.2s
                   all        198       1661      0.438       0.17      0.199      0.146
Speed: 0.6ms preprocess, 6.9ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to C:\Potenup\Drug-Detection-Chatbot\modeling\segment\runs\val


# 테스트 해보기

In [ ]:
import cv2

def enhance_contrast(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    inverted = cv2.bitwise_not(enhanced)   # 흑백 반전
    return cv2.cvtColor(inverted, cv2.COLOR_GRAY2BGR)  # 3채널로 복원

In [ ]:
import cv2
import os
from ultralytics import YOLO

input_dir = "./images/original"
preprocessed_dir = "./images/preprocessed"
output_dir = "./images/results/yolo"

os.makedirs(preprocessed_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(input_dir):
    if fname.lower().endswith((".jpg", ".png", ".jpeg")):
        img_path = os.path.join(input_dir, fname)
        img = cv2.imread(img_path)

        img = enhance_contrast(img)

        # 저장
        out_path = os.path.join(preprocessed_dir, fname)
        cv2.imwrite(out_path, img)

# 전처리된 이미지로 예측 실행
model = YOLO(best_model_path)
results = model.predict(
    source=preprocessed_dir,
    conf=0.5,
    save=True,
    project=output_dir,
    name="",
)

print(f"결과 이미지가 {output_dir} 폴더에 저장되었습니다.")



image 1/3 c:\Potenup\Drug-Detection-Chatbot\modeling\segment\images\preprocessed\test1.jpg: 640x384 5 drugs, 23.3ms
image 2/3 c:\Potenup\Drug-Detection-Chatbot\modeling\segment\images\preprocessed\test2.jpg: 640x384 4 drugs, 30.3ms
image 3/3 c:\Potenup\Drug-Detection-Chatbot\modeling\segment\images\preprocessed\test3.jpg: 640x640 4 drugs, 31.4ms
Speed: 5.0ms preprocess, 28.3ms inference, 3.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Potenup\Drug-Detection-Chatbot\modeling\segment\images\results\yolo\predict5
결과 이미지가 ./images/results/yolo 폴더에 저장되었습니다.
